In [2]:
from source import create_df, preprocess, distributions, popularity_recommendations
import pandas as pd
import sqlite3
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import ipywidgets as ipyw               
from ipywidgets import widgets, interactive_output, interact, interactive, fixed, widget
from IPython.display import display

from sklearn.preprocessing import MinMaxScaler

# **EDA**

In [ ]:
%reload_ext sql
%sql sqlite:///db_movies

In [ ]:
%%sql

SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
HAVING total_ratings <= 1;


SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
ORDER BY watched_movies DESC;

SELECT title, movies.movieId FROM movies
INNER JOIN (SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
HAVING total_ratings <= 1) t1
ON movies.movieId = t1.movieId
ORDER BY title;


SELECT * FROM movies
WHERE genres = "(no genres listed)";

SELECT * FROM (SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
ORDER BY total_ratings) t1
INNER JOIN (SELECT * FROM movies
WHERE genres = "(no genres listed)") t2
ON t1.movieId = t2.movieId;

SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
ORDER BY total_ratings;


SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
ORDER BY watched_movies DESC;


SELECT userId FROM (SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
HAVING watched_movies > 1000);


SELECT rel_yr, title, ROUND(AVG(rating), 3) AS avg_ratings, COUNT(*) total_ratings FROM full_info
GROUP BY rel_yr, title
HAVING rel_yr = 2018
ORDER BY rel_yr DESC, avg_ratings DESC;




In [ ]:
distributions()

In [ ]:
preprocess()

In [ ]:
distributions()

In [ ]:
query = "SELECT userId, COUNT(*) AS watched_movies FROM ratings GROUP BY userId ORDER BY watched_movies DESC"
df = create_df(query)

fig = px.histogram(df, x="watched_movies", nbins=60, template="plotly_white", labels={"watched_movies":"<b>Peliculas vistas</b>"},
                  title="<b>Distribucion Cantidad de peliculas vistas<br>por usuario</b>", color_discrete_sequence=["rgba(109, 176, 223, 0.8)"])

fig.update_layout(title={"x":0.5})

fig.update_traces(hovertemplate="<b>Peliculas vistas</b>=%{x}<br><b>Usuarios</b>=%{y}<extra></extra>")

fig.update_yaxes(title="<b>Usuarios</b>")

fig.show()

In [ ]:
query = "SELECT movieId, COUNT(*) AS total_ratings FROM ratings GROUP BY movieId ORDER BY total_ratings"
df = create_df(query)
# df["conteo"] = df["conteo"]/df["conteo"].sum() 

fig = px.histogram(df, x="total_ratings", nbins=50, template="plotly_white", labels={"total_ratings":"<b>Veces Calificada</b>"},
                  title="<b>Distribucion Cantidad de calificaciones por pelicula</b>", color_discrete_sequence=["rgba(105, 153, 66, 0.8)"])

fig.update_layout(title={"x":0.5})

fig.update_traces(textposition="outside", hovertemplate="<b>Veces Calificada</b>=%{x}<br><b>Peliculas</b>=%{y}<extra></extra>")

fig.update_yaxes(title="<b>Peliculas</b>", )

fig.show()

In [2]:
 popularity_recommendations()

Output()

In [3]:
title = "'71 (2014)"
n = 11

query = "SELECT * FROM movies;"

movies = create_df(query).set_index("movieId")

movies["release_yr"] = movies["title"].str.extract("\((\d{4})\)")

escaler = MinMaxScaler()

index = movies[movies["title"] ==  title].index.values.tolist()[0]

to_corr = pd.concat([movies["release_yr"], movies['genres'].str.get_dummies()], axis=1)

to_corr = pd.DataFrame(escaler.fit_transform(to_corr), columns=to_corr.columns, index=to_corr.index)


movies["similitud"] = to_corr.corrwith(to_corr.loc[index], axis=1)


display(movies.sort_values(by="similitud", ascending=False).iloc[:n,:4])

,title,genres,release_yr,similitud
movieId,,,,
117867,'71 (2014),Action|Drama|Thriller|War,2014,1.000000
107069,Lone Survivor (2013),Action|Drama|Thriller|War,2013,0.999992
174055,Dunkirk (2017),Action|Drama|Thriller|War,2017,0.999929
74795,Green Zone (2010),Action|Drama|Thriller|War,2010,0.999870
57528,Rambo (Rambo 4) (2008),Action|Drama|Thriller|War,2008,0.999706
69481,"Hurt Locker, The (2008)",Action|Drama|Thriller|War,2008,0.999706
27704,Battle Royale 2: Requiem (Batoru rowaiaru II: ...,Action|Drama|Thriller|War,2003,0.998995
46855,Army of Shadows (L'armée des ombres) (1969),Action|Drama|Thriller|War,1969,0.981537
7766,Throne of Blood (Kumonosu jô) (1957),Action|Drama|Thriller|War,1957,0.969613


In [5]:
def filter_content(title, n):
    
    query = "SELECT * FROM movies;"

    movies = create_df(query).set_index("movieId")

    movies["release_yr"] = movies["title"].str.extract("\((\d{4})\)")

    escaler = MinMaxScaler()

    index = movies[movies["title"] ==  title].index.values.tolist()[0]

    to_corr = pd.concat([movies["release_yr"], movies['genres'].str.get_dummies()], axis=1)

    to_corr = pd.DataFrame(escaler.fit_transform(to_corr), columns=to_corr.columns, index=to_corr.index)


    movies["similitud"] = to_corr.corrwith(to_corr.loc[index], axis=1)


    display(movies.sort_values(by="similitud", ascending=False).iloc[:n,:4])

In [8]:
def content_recommendations():
    wid_1 = widgets.Dropdown(options=create_df("SELECT DISTINCT title FROM movies ORDER BY title").values.flatten().tolist(),
                         description="Titulo")
    wid_2 = widgets.BoundedIntText(value=10, min=10, max=60,
                                description="Recomendaciones")


    ui_1 = widgets.VBox([wid_1, wid_2])

    output = interactive_output(filter_content, {"title":wid_1, "n":wid_2})

    display(ui_1, output)

In [9]:
content_recommendations()

Output()